Name: Maurya Vijayaramachandran 

Project: Using pre trained densenet to perform image classification to detect autism. 

Import the libraries

In [ ]:
import os
import torch
import torchvision
import matplotlib.pyplot as plt
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from io import BytesIO
import requests
import torchvision.transforms as T

Import the dataset

In [ ]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od 
od.download("https://www.kaggle.com/datasets/cihan063/autism-image-data", force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mauryav
Your Kaggle Key: ··········


100%|██████████| 229M/229M [00:01<00:00, 154MB/s]


Convert it into a torch tensor

In [ ]:
dataset = ImageFolder('/content/autism-image-data/AutismDataset/consolidated', transform=ToTensor())

Have a look at the image and the class label

In [ ]:
image, label = dataset[5]
print(image.shape, label)
image

torch.Size([3, 281, 180]) 0


tensor([[[0.3294, 0.3294, 0.3255,  ..., 0.3255, 0.3216, 0.3176],
         [0.3294, 0.3294, 0.3255,  ..., 0.3255, 0.3216, 0.3176],
         [0.3294, 0.3255, 0.3255,  ..., 0.3176, 0.3137, 0.3137],
         ...,
         [0.2627, 0.2667, 0.2706,  ..., 0.3804, 0.3804, 0.3765],
         [0.2588, 0.2627, 0.2667,  ..., 0.3843, 0.3804, 0.3765],
         [0.2667, 0.2667, 0.2667,  ..., 0.3804, 0.3725, 0.3765]],

        [[0.6196, 0.6196, 0.6157,  ..., 0.7020, 0.6980, 0.6941],
         [0.6196, 0.6196, 0.6157,  ..., 0.7020, 0.6980, 0.6941],
         [0.6196, 0.6157, 0.6157,  ..., 0.7020, 0.6980, 0.6980],
         ...,
         [0.2510, 0.2549, 0.2588,  ..., 0.6863, 0.6863, 0.6824],
         [0.2471, 0.2510, 0.2549,  ..., 0.6902, 0.6863, 0.6824],
         [0.2549, 0.2549, 0.2549,  ..., 0.6863, 0.6784, 0.6824]],

        [[0.8667, 0.8667, 0.8627,  ..., 0.9412, 0.9373, 0.9333],
         [0.8667, 0.8667, 0.8627,  ..., 0.9412, 0.9373, 0.9333],
         [0.8667, 0.8627, 0.8627,  ..., 0.9412, 0.9373, 0.

In [ ]:
print(dataset.classes)

['Autistic', 'Non_Autistic']


In [ ]:
random_seed = 42
torch.manual_seed(random_seed)

Test train split

In [ ]:
validation_size = 300
test_size = 100
train_size = len(dataset) - validation_size - test_size
train_ds, validation_ds, test_ds = random_split(dataset, [train_size, validation_size, test_size])
len(train_ds), len(validation_ds), len(test_ds)

(2540, 300, 100)

Configure the data loader

In [ ]:
batch_size=32
train_data_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
validation_data_loader = DataLoader(validation_ds, batch_size*2, num_workers=4, pin_memory=True)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Configure the device

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [ ]:
device = get_default_device()
device

device(type='cuda')

Image augmentation layer

In [ ]:
def DiffAugment(x, policy='', channels_first=True):
    if policy:
        if not channels_first:
            x = x.permute(0, 3, 1, 2)
        for p in policy.split(','):
            for f in AUGMENT_FNS[p]:
                x = f(x)
        if not channels_first:
            x = x.permute(0, 2, 3, 1)
        x = x.contiguous()
    return x


def rand_brightness(x):
    x = x + (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) - 0.5)
    return x


def rand_saturation(x):
    x_mean = x.mean(dim=1, keepdim=True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) * 2) + x_mean
    return x


def rand_contrast(x):
    x_mean = x.mean(dim=[1, 2, 3], keepdim=True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) + 0.5) + x_mean
    return x


def rand_translation(x, ratio=0.125):
    shift_x, shift_y = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    translation_x = torch.randint(-shift_x, shift_x + 1, size=[x.size(0), 1, 1], device=x.device)
    translation_y = torch.randint(-shift_y, shift_y + 1, size=[x.size(0), 1, 1], device=x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(x.size(2), dtype=torch.long, device=x.device),
        torch.arange(x.size(3), dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + translation_x + 1, 0, x.size(2) + 1)
    grid_y = torch.clamp(grid_y + translation_y + 1, 0, x.size(3) + 1)
    x_pad = F.pad(x, [1, 1, 1, 1, 0, 0, 0, 0])
    x = x_pad.permute(0, 2, 3, 1).contiguous()[grid_batch, grid_x, grid_y].permute(0, 3, 1, 2)
    return x


def rand_cutout(x, ratio=0.5):
    cutout_size = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    offset_x = torch.randint(0, x.size(2) + (1 - cutout_size[0] % 2), size=[x.size(0), 1, 1], device=x.device)
    offset_y = torch.randint(0, x.size(3) + (1 - cutout_size[1] % 2), size=[x.size(0), 1, 1], device=x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(cutout_size[0], dtype=torch.long, device=x.device),
        torch.arange(cutout_size[1], dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + offset_x - cutout_size[0] // 2, min=0, max=x.size(2) - 1)
    grid_y = torch.clamp(grid_y + offset_y - cutout_size[1] // 2, min=0, max=x.size(3) - 1)
    mask = torch.ones(x.size(0), x.size(2), x.size(3), dtype=x.dtype, device=x.device)
    mask[grid_batch, grid_x, grid_y] = 0
    x = x * mask.unsqueeze(1)
    return x


AUGMENT_FNS = {
    'color': [rand_brightness, rand_saturation, rand_contrast],
    'translation': [rand_translation],
    'cutout': [rand_cutout],
}

Model metrics skeletal code

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class AutismDetectionClassification(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        images = DiffAugment(images, policy='color,translation') 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

Densenet implementation using pretrained weights

In [ ]:
class Net(AutismDetectionClassification):
    def __init__(self, num_classes=2, num_channels=3):
        super().__init__()
        preloaded = torchvision.models.densenet161(pretrained=True)
        self.features = preloaded.features
        self.features.conv0 = nn.Conv2d(num_channels, 96, 7, 2, 3)
        self.classifier = nn.Linear(2208, num_classes, bias=True)
        self.bn = nn.BatchNorm1d(2208)
        del preloaded
        
    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_max_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out

def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    classes = ['autistic', 'non autistic']
    return classes[preds[0].item()]

In [ ]:
Net = Net()

In [ ]:
Net.state_dict

<bound method Module.state_dict of Net(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [ ]:
Net = to_device(Net, device = device)
train_dl = DeviceDataLoader(train_data_loader, device)
val_dl = DeviceDataLoader(validation_data_loader, device)

Evaluation block skeleton

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    best_acc = 0
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        if result['val_acc'] > best_acc:
            torch.save(model.state_dict(), os.path.join('./', 'autism_best_model.pt'))
            best_acc = result['val_acc']
    return history

Hyper parameters of the model

In [ ]:
num_epochs = 400
optimizer = torch.optim.Adam
lr = 0.001

Training the model

In [ ]:
history = fit(num_epochs, lr, Net, train_data_loader, validation_data_loader, optimizer)